# Получаю данные с Plaid

In [1]:
# Вместо «**» были личные данные с Plaid

from plaid import Client

client = Client(client_id='**', secret='**', public_key='**', environment='sandbox')

In [ ]:
payload = {
  "public_key": "**",
  "institution_id": "ins_1",
  "initial_products": ['auth', 'transactions' identity, income, investments, liabilities, and payment_initiation.],
  "options": {
    "webhook": "https://www.genericwebhookurl.com/webhook"
  }
}

In [39]:
import http.client
import mimetypes
conn = http.client.HTTPSConnection("sandbox.plaid.com")
payload = "{\n  \"public_key\": \"**\",\n  \"institution_id\": \"ins_1\",\n  \"initial_products\": [\"auth\" , \"transactions\" , \"assets\"],\n  \"options\": {\n    \"webhook\": \"https://www.genericwebhookurl.com/webhook\"\n  }\n}"
headers = {
  'Content-Type': 'application/json',
  'User-Agent': 'Plaid Postman'
}
conn.request("POST", "/sandbox/public_token/create", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{
  "public_token": "public-sandbox-86cba0af-60e9-47b9-a365-839bd4fa5ee2",
  "request_id": "UTsjvnYE4ExfQHk"
}


In [40]:
import http.client
import mimetypes
conn = http.client.HTTPSConnection("sandbox.plaid.com")
payload = "{\n\t\"client_id\": \"**\",\n\t\"secret\": \"**\",\n\t\"public_token\": \"public-sandbox-86cba0af-60e9-47b9-a365-839bd4fa5ee2\"\n}"
headers = {
  'Content-Type': 'application/json'
}
conn.request("POST", "/item/public_token/exchange", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{
  "access_token": "access-sandbox-87913166-8aef-43fc-9200-53bcae58c291",
  "item_id": "voAorBPBv5UwXxQXp1MdFL971rJkwDuWRpEjQ",
  "request_id": "XFjR07hag84MWHW"
}


In [ ]:
import http.client
import mimetypes
conn = http.client.HTTPSConnection("sandbox.plaid.com")
payload = "{\n    \"institution_id\": \"ins_1\",\n    \"public_key\": \"808a7d442243bf4fb6bd690da3d607\"\n}"
headers = {
  'Content-Type': 'application/json'
}
conn.request("POST", "/institutions/get_by_id", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

In [41]:
import http.client
import mimetypes
conn = http.client.HTTPSConnection("sandbox.plaid.com")
payload = "{\n   \"client_id\": \"**\",\n   \"secret\": \"**\",\n   \"access_tokens\": [\"access-sandbox-87913166-8aef-43fc-9200-53bcae58c291\"],\n   \"days_requested\": 30,\n   \"options\": {\n      \"client_report_id\": \"example_report_id\",\n      \"webhook\": \"https://www.randomdomain.com/webhook_url\",\n      \"user\": {\n        \"client_user_id\": \"example_user_id\",\n        \"first_name\": \"John\",\n        \"middle_name\": \"Deere\",\n        \"last_name\": \"Doe\",\n        \"ssn\": \"123-45-7890\",\n        \"phone_number\": \"3214560987\",\n        \"email\": \"john@doe.com\"\n      }\n   }\n }"
headers = {
  'Content-Type': 'application/json'
}
conn.request("POST", "/asset_report/create", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{
  "asset_report_id": "4eb20fde-bbee-499f-bb1e-fb2f29ec0918",
  "asset_report_token": "assets-sandbox-5b70e9fa-962d-4a15-a30f-09f53a0bef9b",
  "request_id": "fSjFixeWESBmozl"
}


In [ ]:
#Получаем данные в формате json 

import http.client
import mimetypes
conn = http.client.HTTPSConnection("sandbox.plaid.com")
payload = "{\n   \"client_id\": \"**\",\n   \"secret\": \"**\",\n   \"asset_report_token\": \"assets-sandbox-5b70e9fa-962d-4a15-a30f-09f53a0bef9b\"\n}"
headers = {
  'Content-Type': 'application/json'
}
conn.request("POST", "/asset_report/get", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

# Обработка

In [342]:
import pandas as pd

In [343]:
type(data.decode("utf-8"))

str

In [344]:
type(json.loads(data.decode("utf-8")))

dict

In [345]:
#json.loads(data.decode("utf-8"))["report"]["items"][0]["accounts"]

In [346]:
data_save = pd.DataFrame(json.loads(data.decode("utf-8"))["report"]["items"][0]["accounts"])
data_work = data_save.copy()

In [347]:
# Сохраняю исходник

data_save.to_excel("output.xlsx") 

In [377]:
# Проверяю на незаполненные значения

data_work.isnull().sum()
data_work.isna().sum()

In [375]:
# Значения nan были в official_name, смотрю что там

data_work[data_work.isna()["official_name"]]

In [379]:
# Удаляю пользователей у которых нет транзакций

data_work.dropna(axis='index', inplace = True)
data_work = data_work.reset_index(drop = True)

In [380]:
def dict_to_df(dict_data):
    columns = list(dict_data[0].keys())
    data = []
    for i in range(len(dict_data)):
        data.append(list(dict_data[i].values()))
    df = pd.DataFrame(data = data, columns = columns)
    return df

In [386]:
# Информация о транзакциях в нужной форме

def get_trans_data(data, user_id):
    df = dict_to_df(data.loc[user_id]["transactions"].copy())
    return df.loc[:,["account_id","amount", "date", "original_description"]]

In [602]:
# Создаю таблицы Income и Expense

def income_table(trans):
    income = trans[trans["amount"] > 0]
    total = income.groupby("account_id").sum()
    total = total.assign(account_id = ['TOTAL INCOME'])
    income = income.append(total, ignore_index=True)
    return income

def expense_table(trans):
    expense = trans[trans["amount"] < 0]
    total = expense.groupby("account_id").sum()
    total = total.assign(account_id = ['TOTAL EXPENSE'])
    expense = expense.append(total, ignore_index=True)
    return expense

In [604]:
#Объединение таблиц

def union_table(data,user_id):
    income = income_table(get_trans_data(data, user_id))
    expense = expense_table(get_trans_data(data, user_id))
    frames = [income, expense]
    resullt = pd.concat(frames)
    return result

In [608]:
# Таблица в нужном формате для отдельного пользователя

def create_final_by_user(data, user_id):
    total_total = get_trans_data(data, user_id).groupby("account_id").sum()
    total_total = total_total.assign(account_id = ['TOTAL'])
    income = income_table(get_trans_data(data, user_id))
    expense = expense_table(get_trans_data(data, user_id))
    frames = [income, expense]
    resullt = pd.concat(frames)
    resullt = resullt.append(total_total, ignore_index=True)
    return resullt

In [609]:
# Собираю финальную таблицу

def create_final(data):
    l = len(data["account_id"])
    frames = []
    for i in range(len(data["account_id"])):
        frames.append(create_final_by_user(data, i))
    resullt = pd.concat(frames)
    resullt.reset_index(drop = True, inplace=True)
    resullt.fillna('-', inplace=True)
    return resullt
    

In [ ]:
cash_flow = create_final(data_work)

In [611]:
cash_flow.to_excel("cash_flow.xlsx")